# Implementation Based On Selenium Lib

In [13]:
info_list = []

In [14]:
# %%file SeleniumLiepin.py
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

# Initialize variable 
page_number = 3
info_df = pd.DataFrame()

# Set up Edge driver
opt = webdriver.EdgeOptions()   # Create setup object
opt.add_argument('--disable-gpu')
driver = webdriver.Edge(options=opt)

# Wait for element locating
driver.implicitly_wait(10)

# Open the URL
try:
    # url = "https://www.liepin.com"
    url = 'https://www.liepin.com/zhaopin/?city=410&dq=410&pubTime=&currentPage=3&pageSize=40&key=%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86%E5%B7%A5%E7%A8%8B%E5%B8%88&suggestTag=&workYearCode=0&compId=&compName=&compTag=&industry=&salary=&jobKind=&compScale=&compKind=&compStage=&eduLevel=&otherCity=&sfrom=search_job_pc&ckId=01q2gz6bfx7lccu04cvnflv403m991qb&scene=page&skId=01q2gz6bfx7lccu04cvnflv403m991qb&fkId=01q2gz6bfx7lccu04cvnflv403m991qb&suggestId='
    driver.get(url)

    # # Type and search job position
    # # Locate the `input` element and entry key word
    # driver.find_element(By.XPATH, "//input[@placeholder='搜索职位/公司/内容关键词']").send_keys("自然语言处理工程师")
    # # Locate the search button element and click the button
    # driver.find_element(By.XPATH, "//span[text()='搜索']").click()

    # Switch to recruitment page
    driver.switch_to.window(driver.window_handles[-1])

    while True:
        tmp_data_list = []
        tmp_data_df = pd.DataFrame()

        # Wait for page loading to be completed
        # while not driver.execute_script("document.readyState == 'complete'"):
        #     time.sleep(1)
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@id='lp-search-job-box']/div[3]/section[1]/div[1]/div[40]")))
        eles = driver.find_elements(By.XPATH, "//div[@id='lp-search-job-box']/div[3]/section[1]/div[1]/div")

        # Extracte job information
        for ele in eles:
            tmp_data_dict = dict()
            try:
                tmp_data_dict['position'] = ele.find_element(By.XPATH, "./div/div[1]/div/a/div[1]/div/div[1]").text
            except NoSuchElementException:
                tmp_data_dict['position'] = 'N/A'  
            try:
                tmp_data_dict['city'] = ele.find_element(By.XPATH, "./div/div[1]/div/a/div[1]/div/div[2]/span[2]").text  
            except NoSuchElementException:
                tmp_data_dict['city'] = 'N/A'  
            try:
                tmp_data_dict['payment'] = ele.find_element(By.XPATH, "./div/div[1]/div/a/div[1]/span[@class='jsx-2693574896 job-salary']").text
            except NoSuchElementException:
                tmp_data_dict['payment'] = 'N/A'  
            try:
                tmp_data_dict['enterprise'] = ele.find_element(By.XPATH, "./div/div[1]/div/div/div/span").text
            except NoSuchElementException:
                tmp_data_dict['enterprise'] = 'N/A'  
            try:
                tmp_data_dict['requirement-list'] = ele.find_element(By.XPATH, "./div/div[1]/div/a/div[2]").text
            except NoSuchElementException:
                tmp_data_dict['requirement-list'] = 'N/A'  
            try:
                tmp_data_dict['CPprofile-list'] = ele.find_element(By.XPATH, "./div/div[1]/div/div/div/div[2]").text
            except NoSuchElementException:
                tmp_data_dict['CPprofile-list'] = 'N/A'  

            # # Wait for element to be visible and scroll to the visible area
            # driver.execute_script('arguments[0].scrollIntoView();', ele)
            # ele = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(ele))
            
            # Open job information detail page
            ele.click()
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(60)

            # Extract job information detail
            # Job description
            try:
                tmp_data_dict['job-detail'] = driver.find_element(By.XPATH, "//dd[@data-selector='job-intro-content']").text   # responsibilities and requirement-detail
            except NoSuchElementException:
                tmp_data_dict['job-detail'] = 'N/A'  
            # Profile-detail
            try:
                tmp_data_dict['enterprise-detail'] = driver.find_element(By.XPATH, "//div[@class='paragraph-box']//div[1]").text   # enterprise profile-detail
            except NoSuchElementException:
                tmp_data_dict['enterprise-detail'] = 'N/A'  
            tmp_data_list.append(tmp_data_dict)
            info_list.append(tmp_data_dict)

            # Return overview page
            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
            # time.sleep(5)

        # Save current page data
        with open(f'tmp_data_dir/{page_number}.json', 'w') as ftd:
            json.dump(tmp_data_list, ftd, ensure_ascii=False)
        tdf = tmp_data_df.from_records(tmp_data_list)
        tdf.to_csv(f'tmp_data_dir/{page_number}.csv', index=False)

        # Go to next page
        next_button = driver.find_element(By.XPATH, "//li[@title='Next Page']//button[1]")
        if next_button.get_attribute('disabled') == 'true':
            break
        else:
            next_button.click()
            page_number += 1
            time.sleep(60)

    # Close the browser
    driver.quit()

# Capture exceptions and exit the browser
except Exception as e:
    print(e)
    driver.quit()


# Save information data
with open('job1.json', 'w') as f:
    json.dump(info_list, f, ensure_ascii=False)

df = info_df.from_records(info_list)
df.to_csv('job1.csv', index=False)

In [12]:
import re

print(len(info_list))
# print(type(info_list[2]['job-detail'].encode('unicode_escape').decode('utf-8')))
# pprint(info_list[60:])

# s = info_list[60:]
# def find_incompatible_characters(string):
#     # 定义匹配非法字符的正则表达式
#     pattern = re.compile(r'[^\x09\x0A\x0D\x20-\uD7FF\uE000-\uFFFD\u10000-\u10FFFF]')

#     # 查找非法字符
#     incompatible_characters = re.findall(pattern, string)

#     return incompatible_characters

# # 示例用法
# string = "Hello, <world>!"
# incompatible_chars = find_incompatible_characters(s)
# if incompatible_chars:
#     print("字符串中存在与XML不兼容的字符：", incompatible_chars)
# else:
#     print("字符串中不存在与XML不兼容的字符")

0


# Implementation Based On Requests Lib

In [3]:
# %%file liepin.py

# Code to scrape job information from the website goes here

import requests
from bs4 import BeautifulSoup

# Prepera request
while True:
    url = "https://jdjyw.jlu.edu.cn/portal/jyzp/recruit/list?type=1&pageNo=4&pageSize=12&filterParams%5BcompanyNatureCategory%5D=&campusArea=&filterParams%5Bdate%5D=&keyword="
    # url = "https://www.liepin.com/zhaopin/?inputFrom=www_index&workYearCode=0&key=%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86%E5%B7%A5%E7%A8%8B%E5%B8%88&scene=input&ckId=3b5ozfzgjg4lllej2tlhw59ck4r860p8&dq="
    # headers = {
    #     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    #     "Accept-Encoding": "gzip, deflate, br",
    #     "Accept-Language": "zh-CN,zh;q=0.9",
    #     "Cookie": "__gc_id=f22ffae0bcc3463da4baaebd39565220; __uuid=1691135429066.26; _ga=GA1.1.1902532824.1691135430; __tlog=1691470573010.41%7C00000000%7C00000000%7C00000000%7C00000000; XSRF-TOKEN=bFLGHjv6TSO1FP0pFVQcxQ; Hm_lvt_a2647413544f5a04f00da7eee0d5e200=1691212344,1691294573,1691458626,1691470573; acw_tc=2760829a16914742554304576e46e7ebfa068095afda8b4d9c185c66cd426e; __session_seq=4; __uv_seq=12; Hm_lpvt_a2647413544f5a04f00da7eee0d5e200=1691474255; _ga_54YTJKWN86=GS1.1.1691474240.11.1.1691474255.0.0.0",
    #     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    #     # sec-ch-ua: "Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"
    # }

    response = requests.get(url, )
    soup = BeautifulSoup(response.content)
    soup

    # Save page source
    # with open('zhaopin.html', 'wb') as fr:
    #     fr.write(response.content)


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/static/jlu-favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/static/portal/v2/style/reset.css" rel="stylesheet"/>
<link href="/static/portal/v2/style/all-new.css" rel="stylesheet"/>
<link href="/static/portal/v2/style/jyzp_side_calendar.css" rel="stylesheet"/>
<title>校园招聘列表 - 吉林大学就业网</title>
<script type="text/javascript">var ctx = '/portal', ctxStatic='/static';</script>
<style>
      .form-group {
        display: flex;
        justify-content: flex-start;
      }

      .form-group > label {
        margin: 0 4px 0 16px;
      }
    </style>
</head>
<body>
<!-- 首页头部 -->
<!-- 首页头部 -->
<div class="m-header-sub">
<section>
<div>
<h1><a href="/"><img alt="" src="/static/portal/v2/image/logo-main.png"/></a></h1>
<h3>就业招聘</h3>
</div>
<ul>
<li><a href="/portal/jyzp">首页</a></li>
<

In [4]:
help(soup)

Help on BeautifulSoup in module bs4 object:

class BeautifulSoup(bs4.element.Tag)
 |  BeautifulSoup(markup='', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)
 |  
 |  A data structure representing a parsed HTML or XML document.
 |  
 |  Most of the methods you'll call on a BeautifulSoup object are inherited from
 |  PageElement or Tag.
 |  
 |  Internally, this class defines the basic interface called by the
 |  tree builders when converting an HTML/XML document into a data
 |  structure. The interface abstracts away the differences between
 |  parsers. To write a new tree builder, you'll need to understand
 |  these methods as a whole.
 |  
 |  These methods will be called by the BeautifulSoup constructor:
 |    * reset()
 |    * feed(markup)
 |  
 |  The tree builder may call these methods from its feed() implementation:
 |    * handle_starttag(name, attrs) # See note about return value
 |    * handle_endtag(n

In [7]:
# Authenticate if necessary
# ...

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")
print(soup)

<!DOCTYPE html>

<html lang="en">
<head>
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-54YTJKWN86"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());

        gtag('config', 'G-54YTJKWN86');
    </script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>【招聘信息_人才网招聘信息】-猎聘</title>
<meta content="招聘信息,人才信息,人才网招聘信息" name="keywords"/>
<meta content="猎聘招聘信息网提供真实的人才网招聘信息,每日上万高薪职位招聘信息发布,是专业人才、精英人才获取招聘信息的专业平台。找2023招聘信息,就来人才网招聘信息频道。" name="description"/>
<meta content="pc" name="applicable-device"/>
<meta content="no-transform" http-equiv="Cache-Control">
<meta content="no-siteapp" http-equiv="Cache-Control"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="//concat.lietou-static.com" rel="dns-prefetch"/>
<link href="//image0.li

In [ ]:

# Extract job information
job_list = []
# ...

# Example code to print the scraped information
for job in job_list:
    print(job.title)
    print(job.company)
    print(job.location)
    print(job.salary)
    print(job.description)
    print()



